In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import copy
import json
import sys
import re
import os
from pathlib import Path
from itertools import chain
from functools import partial
from io import StringIO

import blpapi
# import clarion
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import BDay
# from clarion import positions

from api.gui.base import View, Tabs, Store, store
from api.gui.market_monitor import (
    CurvesViewer, 
    Pivot, 
    get_issuers_curves,
    get_issuer_curve
)
from api.gui.max_diff import LowestFieldCurve
from api.gui.visual import Visual
from api.gui.clarion_comp import Comp
from api.gui.cdx import CDXComp
from api.blpw import BlpQuery
from api.bbg import BBGField, BBGOverrides
from api.data.base import data_path, tables
from api.data.curves import curves
from api.data.utils import add_pcs, parse_tenor
from api.pricing.cds import get_isda_cds
from api.data.processing import (
    apply_function, 
    add_years_to_maturity, 
    add_maturity_bucket,
    add_custom_rating,
    add_rating_bucket,
    add_geo_area
)
from api.gui.market_monitor_h import get_bond_ref, get_bday, DatePicker
from api.bbg import BBGRequestParams, BBGField, BBGSecurity, BBGOverrides

In [3]:
dt_fmt = '%Y-%m-%d'
bbg_dt_fmt = '%Y%m%d'
bq = BlpQuery(timeout=50000).start()

In [4]:
bond_ref = add_maturity_bucket(
    add_years_to_maturity(tables.bonds_reference),
    maturity_bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 20, 25, 30, 35, 40, 45, 50, 60]
)
bond_ref = add_rating_bucket(add_custom_rating(bond_ref))
bond_ref = add_geo_area(bond_ref, more_cols={'name': 'country_name'})

In [5]:
cds_ref = pd.read_csv(data_path / 'cds_reference.csv')

In [6]:
# store = Store()
store.data['bond_ref'] = copy.deepcopy(bond_ref)
store.data['cds_ref'] = copy.deepcopy(cds_ref)
# store.data['issuers_curves'] = copy.deepcopy(issuers_curves)

In [ ]:
app = Tabs(
    {
        'curves': CurvesViewer(build=False),
        'pivot': Pivot(build=False),
        'max diff': LowestFieldCurve(build=False),
        'visual': Visual(build=False),
        'clarion comp': Comp(build=False),
        'cdx': CDXComp(build=False)
    },
    store=store
)
app

In [ ]:
pcs='MSG1'

In [ ]:
cds_issuer_mapping = {
    'BRAZIL': 'BRAZIL',
    'CHILE': 'CHILE',
    'COLOM': 'COLOM',
    'EGYPT': 'EGYPT',
    'SAUDI': 'SAUDI',
    'MEX': 'MEX',
    'MOROC': 'MOROC',
    'PERU': 'PERU',
    'CHINA': 'CHINAGOV',
    'SOAF': 'REPSOU',
    'TURKEY': 'TURKEY'
}

cds_bbg = pd.concat(
    [
        (cds_data['Issuer'].map(cds_issuer_mapping) + f' CDS ' + cds_data[
            'Ccy'] + f' SR 5Y D14 {f"{pcs} " if pcs else ""}Curncy').rename('ticker'),
        cds_data['Expiry/ Maturity'].dt.strftime(bbg_dt_fmt).rename('mat'),
        cds_data['Description'].str.split(' ').str[2].astype(float).rename('coupon')
    ], axis=1
)

bbg_securities = [
    BBGSecurity(
        id=row['ticker'],
        overrides=[
            BBGOverrides(
                MATURITY=row['mat'],
                SW_SPREAD=row['coupon']
            )
        ]
    ) for index, row in cds_bbg.iterrows()]
bbg_fields = [
    # if I donwload cds_quoted_price this is not always accurate for cds that quote upfront. if I download 100 - upfront_last I don't have this problem
    # BBGField("cds_quoted_price"),
    BBGField("upfront_last"),
]

In [ ]:
cds_quoted_price = bq.bdpx(BBGRequestParams(securities=bbg_securities, fields=bbg_fields))

In [ ]:
sec1_dd = v.Autocomplete(
    v_model=None,
    items=['cds', 'bond'],
    label='type1',
    dense=True,
    outlined=True,
)

sec2_dd = v.Autocomplete(
    v_model=None,
    items=['cds', 'bond'],
    label='type2',
    dense=True,
    outlined=True,
)


name1_ac = v.Autocomplete(
    v_model=None,
    items=None,
    label='security1',
    dense=True,
    outlined=True,
    class_='ml-2'
)

name2_ac = v.Autocomplete(
    v_model=None,
    items=None,
    label='security2',
    dense=True,
    outlined=True,
    class_='ml-2'
)


date_pk = DatePicker(
    value=None,
    label='start date',
    outlined=True,
)

In [ ]:
sec1_dd = v.Autocomplete(
    v_model=None,
    items=['cds', 'bond'],
    label='type1',
    dense=True,
    outlined=True,
)

sec2_dd = v.Autocomplete(
    v_model=None,
    items=['cds', 'bond'],
    label='type2',
    dense=True,
    outlined=True,
)


name1_ac = v.Autocomplete(
    v_model=None,
    items=None,
    label='security1',
    dense=True,
    outlined=True,
    class_='ml-2'
)

name2_ac = v.Autocomplete(
    v_model=None,
    items=None,
    label='security2',
    dense=True,
    outlined=True,
    class_='ml-2'
)


date_pk = DatePicker(
    value=None,
    label='start date',
    outlined=True,
)

pcs1_tf = v.TextField(
    v_model='BGN',
    label='pcs1',
    clearable=True,
    dense=True,
    outlined=True,
    class_='ml-2'
)

pcs2_tf = v.TextField(
    v_model='BGN',
    label='pcs1',
    clearable=True,
    dense=True,
    outlined=True,
    class_='ml-2'
)

notional1_tf = v.TextField(
    v_model=None,
    label='notional1',
    clearable=True,
    dense=True,
    outlined=True,
    type="number",
    class_='ml-2'
)

notional2_tf = v.TextField(
    v_model=None,
    label='notional2',
    clearable=True,
    dense=True,
    outlined=True,
    type="number",
    class_='ml-2'
)

price1_tf = v.TextField(
    v_model=None,
    label='price1',
    clearable=True,
    dense=True,
    outlined=True,
    type="number",
    class_='ml-2'
)

price2_tf = v.TextField(
    v_model=None,
    label='price2',
    clearable=True,
    dense=True,
    outlined=True,
    type="number",
    class_='ml-2'
)

spread1_tf = v.TextField(
    v_model=None,
    label='spread1',
    clearable=True,
    dense=True,
    outlined=True,
    type="number",
    class_='ml-2'
)

spread2_tf = v.TextField(
    v_model=None,
    label='spread2',
    clearable=True,
    dense=True,
    outlined=True,
    type="number",
    class_='ml-2'

)

inc_tf = v.TextField(
    v_model=0.25,
    label='increment',
    clearable=True,
    dense=True,
    outlined=True,
    type="number",
    class_='ml-2'
)

In [ ]:
import sys

In [ ]:
self = sys.modules[__name__]

In [ ]:
SEP = ' -> '

In [ ]:
def on_change_update_items(widget, event, payload, widget2):
    if payload == 'bond':
        items = (self.bond_ref['name'] + SEP + self.bond_ref['currency']).sort_values()
    else:
        items = (self.cds_ref['name'] + SEP + self.cds_ref['currency']).sort_values()
    widget2.items = [*items]
    # for widget_name in ['name1_ac', 'name2_ac']:
    #     widget2 = getattr(self, widget_name)
    #     widget2.items = [*self.bond_ref[widget.v_model].sort_values()]

In [ ]:
sec1_dd.on_event(
    'change',  
    partial(on_change_update_items, widget2=self.name1_ac)
)
sec2_dd.on_event(
    'change',  
    partial(on_change_update_items, widget2=self.name2_ac)
)

In [ ]:
w.VBox(
    [
        w.HBox(
            [
                sec1_dd, 
                name1_ac,
                pcs1_tf,
                notional1_tf,
                price1_tf,
                spread1_tf,
                date_pk.view
            ]
              ), 
        w.HBox(
            [
                sec2_dd, 
                name2_ac,
                pcs2_tf,
                notional2_tf,
                price2_tf,
                spread2_tf,
                inc_tf
            ]
        )
    ]
)

In [ ]:
sec1_dd.fire_event('change', 'cds')
sec2_dd.fire_event('change', 'bond')
sec1_dd.v_model = 'cds'
sec2_dd.v_model = 'bond'
name1_ac.v_model = f'COLOM CDS USD SR 5Y D14 Curncy{SEP}USD'
name2_ac.v_model = f'COLOM 3 7/8 02/15/61{SEP}USD'
pcs1_tf.v_model = 'MSG1'
pcs2_tf.v_model = 'BGN'
notional1_tf.v_model = 50000000
notional2_tf.v_model = -85000000
price1_tf.v_model = 49.37
price2_tf.v_model = 89.51
spread1_tf.v_model = 350
spread2_tf.v_model = 500
inc_tf.v_model = 25
date_pk.dp.v_model = '2022-10-19'

In [ ]:
sec1 = sec1_dd.v_model
sec2 = sec2_dd.v_model
name1 = name1_ac.v_model
name2 = name2_ac.v_model
pcs1 = pcs1_tf.v_model
pcs2 = pcs2_tf.v_model
notional1 = notional1_tf.v_model
notional2 = notional2_tf.v_model
price1 = price1_tf.v_model
price2 = price2_tf.v_model
spread1 = spread1_tf.v_model
spread2 = spread2_tf.v_model
date = date_pk.date
inc = inc_tf.v_model

In [ ]:
def get_all_prices(self):
    sec1 = sec1_dd.v_model
    sec2 = sec2_dd.v_model
    name1 = name1_ac.v_model
    name2 = name2_ac.v_model
    pcs1 = pcs1_tf.v_model
    pcs2 = pcs2_tf.v_model
    notional1 = notional1_tf.v_model
    notional2 = notional2_tf.v_model
    price1 = price1_tf.v_model
    price2 = price2_tf.v_model
    spread1 = spread1_tf.v_model
    spread2 = spread2_tf.v_model
    date = date_pk.date
    inc = inc_tf.v_model
    
    p1 = get_bond_prices(name=name1, spreads=inc1, date=date, pcs=pcs1) if sec1 == 'bond' \
        else get_cds_prices(name=name1, spreads=inc1, date=date, pcs=pcs1)
    
    p2 = get_bond_prices(name=name2, spreads=inc2, date=date, pcs=pcs2) if sec1 == 'bond' \
        else get_cds_prices(name=name2, spreads=inc2, date=date, pcs=pcs2)
    
    return p1, p2

In [ ]:
from api.utils import get_securities

In [ ]:
def make_increments(start, step, up, down):
    return [*range(start - step * down, start + step, step), *range(start, start  + (step * up + 1), step)]
inc1 = make_increments(spread1, inc, 12, 6)
inc2 = make_increments(spread2, inc, 6, 8)

In [ ]:
inc1 = make_increments(spread1, inc, 12, 6)
inc2 = make_increments(spread2, inc, 6, 8)

In [ ]:
def get_bond_prices(name, spreads, date, pcs):
    price_field = "yas_bond_px"
    name_ = name.split(SEP)[0]
    ticker = self.bond_ref.query(f"name == '{name_}'").squeeze()['security']
    ticker = get_securities(ticker, pcs=pcs, sep='@')
    prices = bq.bdpm(
        args=[
            dict(
                security=ticker,
                fields=[price_field],
                overrides={'yas_zspread': spreads}
            ),
        ],
        common_overrides={'user_local_trade_date': f"{date:{bbg_dt_fmt}}"}
    ).sort_values('yas_zspread')
    return prices.rename({price_field: 'price'}, axis=1)

In [ ]:
get_bond_prices(name=name2, spreads=inc2, date=date, pcs=pcs2)

In [ ]:
def get_cds_prices(name, spreads, date, pcs):
    price_field = "upfront_last"
    name_ = name.split(SEP)[0]
    ticker = get_securities(name_, pcs=pcs)
    # return ticker
    prices = bq.bdpm(
        args=[
            dict(
                security=ticker,
                fields=[price_field],
                overrides={"cds_flat_spread": spreads}
            ),
        ],
        # common_overrides={'SW_CURVE_DT': f"{date:{bbg_dt_fmt}}"}
    )
    prices['price'] = 100 - prices[price_field]
    return prices.sort_values('cds_flat_spread')

In [ ]:
dd = get_cds_prices(name=name1, spreads=inc1, date=date, pcs=pcs1)

In [22]:
from api.gui.scen_grid import Grid, SEP, make_increments

In [125]:
self = Grid()
self

In [126]:
self.sec1_dd.fire_event('change', 'cds')
self.sec2_dd.fire_event('change', 'bond')
self.sec1_dd.v_model = 'cds'
self.sec2_dd.v_model = 'bond'
self.name1_ac.v_model = f'COLOM CDS USD SR 5Y D14 Curncy{SEP}USD'
self.name2_ac.v_model = f'COLOM 3 7/8 02/15/61{SEP}USD'
self.pcs1_tf.v_model = 'MSG1'
self.pcs2_tf.v_model = None
self.notional1_tf.v_model = -85000000
self.notional2_tf.v_model = 50000000
self.price1_tf.v_model = 89.51
self.price2_tf.v_model = 49.37
self.spread1_tf.v_model = 350
self.spread2_tf.v_model = 500
self.inc_tf.v_model = 25
self.date_pk.dp.v_model = '2022-10-19'

In [80]:
sec1 = self.sec1_dd.v_model
sec2 = self.sec2_dd.v_model
name1 = self.name1_ac.v_model
name2 = self.name2_ac.v_model
pcs1 = self.pcs1_tf.v_model
pcs2 = self.pcs2_tf.v_model
notional1 = self.notional1_tf.v_model
notional2 = self.notional2_tf.v_model
price1 = self.price1_tf.v_model
price2 = self.price2_tf.v_model
spread1 = self.spread1_tf.v_model
spread2 = self.spread2_tf.v_model
date = self.date_pk.date
inc = self.inc_tf.v_model

inc1 = make_increments(spread1, inc, 12, 6)
inc2 = make_increments(spread2, inc, 6, 8)

In [116]:
grid.style.format('{:.2f}').format_index('{:.2f}')

ValueError: Unknown format code 'f' for object of type 'str'